In [ ]:
import os
import json
import cv2
import numpy as np

# Root directory of your dataset
main_folder = 'path_to_main_folder'

# Output where you want to save masks
output_mask_dir = 'output_masks'
os.makedirs(output_mask_dir, exist_ok=True)

# Traverse all subfolders
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    annotation_folder = os.path.join(subfolder_path, 'annotations')
    image_folder = os.path.join(subfolder_path, 'images')

    # Iterate over each annotation file
    for json_file in os.listdir(annotation_folder):
        if not json_file.endswith('.json'):
            continue

        with open(os.path.join(annotation_folder, json_file)) as f:
            annotations = json.load(f)

        for item in annotations.values():
            filename = item['filename']
            regions = item['regions']

            # Look for image
            found_image = None
            for sub_img_folder in os.listdir(image_folder):
                possible_path = os.path.join(image_folder, sub_img_folder, filename)
                if os.path.exists(possible_path):
                    found_image = possible_path
                    break

            if not found_image:
                continue

            img = cv2.imread(found_image)
            h, w = img.shape[:2]
            mask = np.zeros((h, w), dtype=np.uint8)

            # Draw each region as white on black mask
            for region in item['regions']:
                shape = region['shape_attributes']

                if shape['name'] == 'polygon':
                    points = np.array(list(zip(shape['all_points_x'], shape['all_points_y'])), dtype=np.int32)
                    cv2.fillPoly(mask, [points], color=255)

                elif shape['name'] == 'rect':
                    x, y, w_, h_ = shape['x'], shape['y'], shape['width'], shape['height']
                    cv2.rectangle(mask, (x, y), (x + w_, y + h_), 255, -1)

            # Save mask
            rel_path = os.path.relpath(found_image, main_folder).replace(os.sep, '_')
            mask_path = os.path.join(output_mask_dir, f"{os.path.splitext(rel_path)[0]}_mask.png")
            cv2.imwrite(mask_path, mask)


Classes: ['fake', 'real']


In [17]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128) 
        self.fc2 = nn.Linear(128, 2)  

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.softmax(x, dim=1) 
        return x
    
